# 최적 모델 찾기
- [최적 모델 찾기](https://snowdeer.github.io/machine-learning/2018/01/09/find-best-model/)
- 모델 생성 시 seed = 42 ~ 50일 때, 각각의 best모델에서 data seed = 1~50에서의 평균 정확도
- 모델 생성 시 seed = 42, 평균 정확도 : 82.6
- 모델 생성 시 seed = 43, 평균 정확도 : 81.6
- 모델 생성 시 seed = 44, 평균 정확도 : 82.5
- 모델 생성 시 seed = 45, 평균 정확도 : 82.5
- 모델 생성 시 seed = 46, 평균 정확도 : 81.6
- 모델 생성 시 seed = 47, 평균 정확도 : 81.8
- 모델 생성 시 seed = 48, 평균 정확도 : 81.1
- 모델 생성 시 seed = 49, 평균 정확도 : 82.3
- 모델 생성 시 seed = 50, 평균 정확도 : 80

# data_seed, model_seed = 42

In [6]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


import random
# ----------------------
seed_num = 42
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# ---------------------
seed_num = 42 
# ---------------------
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'seed42-{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_acc', patience=50, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
37/37 [==============================] - 5s 64ms/step - loss: 0.6757 - acc: 0.5984 - val_loss: 0.6654 - val_acc: 0.6177

Epoch 00001: val_loss improved from inf to 0.66544, saving model to ./model/seed42-01-0.6654.hdf5
Epoch 2/500
37/37 [==============================] - 2s 52ms/step - loss: 0.6690 - acc: 0.6089 - val_loss: 0.6482 - val_acc: 0.6177

Epoch 00002: val_loss improved from 0.66544 to 0.64818, saving model to ./model/seed42-02-0.6482.hdf5
Epoch 3/500
37/37 [==============================] - 2s 52ms/step - loss: 0.6000 - acc: 0.6713 - val_loss: 0.5525 - val_acc: 0.7354

Epoch 00003: val_loss improved from 0.64818 to 0.55246, saving model to ./model/seed42-03-0.5525.hdf5
Epoch 4/500
37/37 [==============================] - 2s 52ms/step - loss: 0.4867 - acc: 0.7793 - val_loss: 0.5088 - val_acc: 0.7613

Epoch 00004: val_loss improved from 0.55246 to 0.50875, saving model to ./model/seed42-04-0.5088.hdf5
Epoch 5/500
37/37 [==============================] - 2s 52ms/ste

37/37 [==============================] - 2s 53ms/step - loss: 0.0637 - acc: 0.9875 - val_loss: 1.1007 - val_acc: 0.7264

Epoch 00044: val_loss did not improve from 0.50213
Epoch 45/500
37/37 [==============================] - 2s 53ms/step - loss: 0.0626 - acc: 0.9871 - val_loss: 1.1301 - val_acc: 0.7290

Epoch 00045: val_loss did not improve from 0.50213
Epoch 46/500
37/37 [==============================] - 2s 52ms/step - loss: 0.0666 - acc: 0.9860 - val_loss: 1.0977 - val_acc: 0.7309

Epoch 00046: val_loss did not improve from 0.50213
Epoch 47/500
37/37 [==============================] - 2s 52ms/step - loss: 0.0645 - acc: 0.9860 - val_loss: 1.1112 - val_acc: 0.7264

Epoch 00047: val_loss did not improve from 0.50213
Epoch 48/500
37/37 [==============================] - 2s 52ms/step - loss: 0.0662 - acc: 0.9853 - val_loss: 1.1206 - val_acc: 0.7283

Epoch 00048: val_loss did not improve from 0.50213
Epoch 49/500
37/37 [==============================] - 2s 52ms/step - loss: 0.0605 - acc:

In [9]:
from keras.models import load_model
best_model = load_model('./model/seed42-05-0.5021.hdf5') 

dic_42={}
for seed in range(0, 50):
    random.seed(seed)

    x = np.load('/project/LSH/x_(7727,10,4069).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    idx = list(range(len(x)))
    random.shuffle(idx)

    i = round(x.shape[0]*0.8)
    X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = x[idx[i:],:,:], y[idx[i:]]
    
    pred = best_model.predict(X_test)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    dic_42[seed]=acc
    print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')

정확도 :0.8194174757281554, seed_num = 0
정확도 :0.8362459546925566, seed_num = 1
정확도 :0.8207119741100324, seed_num = 2
정확도 :0.8388349514563107, seed_num = 3
정확도 :0.8252427184466019, seed_num = 4
정확도 :0.8401294498381877, seed_num = 5
정확도 :0.8103559870550162, seed_num = 6
정확도 :0.8155339805825242, seed_num = 7
정확도 :0.827831715210356, seed_num = 8
정확도 :0.8317152103559871, seed_num = 9
정확도 :0.8265372168284789, seed_num = 10
정확도 :0.8310679611650486, seed_num = 11
정확도 :0.8349514563106796, seed_num = 12
정확도 :0.8343042071197411, seed_num = 13
정확도 :0.8381877022653722, seed_num = 14
정확도 :0.8336569579288026, seed_num = 15
정확도 :0.8330097087378641, seed_num = 16
정확도 :0.8174757281553398, seed_num = 17
정확도 :0.8271844660194175, seed_num = 18
정확도 :0.8388349514563107, seed_num = 19
정확도 :0.8284789644012945, seed_num = 20
정확도 :0.8142394822006472, seed_num = 21
정확도 :0.8226537216828479, seed_num = 22
정확도 :0.8090614886731392, seed_num = 23
정확도 :0.8317152103559871, seed_num = 24
정확도 :0.8466019417475729, seed_num = 

In [16]:
# seed = 42의 정확도 df 만들고 평균 확인 => 82.6
df_42 = pd.DataFrame.from_dict(dic_42, orient='index')
df_42.mean()

0    0.826731
dtype: float64

# seed별 best model 저장

In [43]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


import random
# ----------------------
seed_num = 50
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# ---------------------
seed_num = 50 
# ---------------------
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'seed'f'{seed_num}''-{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_acc', patience=30, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
37/37 [==============================] - 5s 64ms/step - loss: 0.6754 - acc: 0.5949 - val_loss: 0.6753 - val_acc: 0.5970

Epoch 00001: val_loss improved from inf to 0.67535, saving model to ./model/seed50-01-0.6753.hdf5
Epoch 2/500
37/37 [==============================] - 2s 52ms/step - loss: 0.6743 - acc: 0.6031 - val_loss: 0.6701 - val_acc: 0.5970

Epoch 00002: val_loss improved from 0.67535 to 0.67009, saving model to ./model/seed50-02-0.6701.hdf5
Epoch 3/500
37/37 [==============================] - 2s 52ms/step - loss: 0.6285 - acc: 0.6398 - val_loss: 0.5877 - val_acc: 0.7025

Epoch 00003: val_loss improved from 0.67009 to 0.58767, saving model to ./model/seed50-03-0.5877.hdf5
Epoch 4/500
37/37 [==============================] - 2s 52ms/step - loss: 0.5010 - acc: 0.7681 - val_loss: 0.5157 - val_acc: 0.7536

Epoch 00004: val_loss improved from 0.58767 to 0.51568, saving model to ./model/seed50-04-0.5157.hdf5
Epoch 5/500
37/37 [==============================] - 2s 52ms/ste

In [44]:
# ----------------------------
seed_num = 50
# ----------------------------

from keras.models import load_model
best_model = load_model('./model/seed50-04-0.5157.hdf5') # 수동으로 확인

dic={}
for seed in range(0, 50):
    random.seed(seed)

    x = np.load('/project/LSH/x_(7727,10,4069).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    idx = list(range(len(x)))
    random.shuffle(idx)

    i = round(x.shape[0]*0.8)
    X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = x[idx[i:],:,:], y[idx[i:]]
    
    pred = best_model.predict(X_test)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    dic[seed]=acc
    print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')
    
df = pd.DataFrame.from_dict(dic, orient='index')
print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')

정확도 :0.7870550161812297, seed_num = 0
정확도 :0.8019417475728156, seed_num = 1
정확도 :0.7954692556634304, seed_num = 2
정확도 :0.8090614886731392, seed_num = 3
정확도 :0.7954692556634304, seed_num = 4
정확도 :0.8025889967637541, seed_num = 5
정확도 :0.7954692556634304, seed_num = 6
정확도 :0.7877022653721683, seed_num = 7
정확도 :0.8084142394822007, seed_num = 8
정확도 :0.8006472491909385, seed_num = 9
정확도 :0.8129449838187702, seed_num = 10
정확도 :0.8129449838187702, seed_num = 11
정확도 :0.8064724919093851, seed_num = 12
정확도 :0.8116504854368932, seed_num = 13
정확도 :0.8045307443365696, seed_num = 14
정확도 :0.8019417475728156, seed_num = 15
정확도 :0.8116504854368932, seed_num = 16
정확도 :0.7902912621359224, seed_num = 17
정확도 :0.8006472491909385, seed_num = 18
정확도 :0.8019417475728156, seed_num = 19
정확도 :0.8071197411003236, seed_num = 20
정확도 :0.7909385113268609, seed_num = 21
정확도 :0.7941747572815534, seed_num = 22
정확도 :0.7922330097087379, seed_num = 23
정확도 :0.7967637540453074, seed_num = 24
정확도 :0.8148867313915857, seed_num =